In [7]:
import spacy
from spacy import displacy

nlp = spacy.load('pt')

# Named Entity Recognition

Named Entities supported by Spacy: https://spacy.io/api/annotation#named-entities

In [8]:
doc = nlp('Maria está se mudando para Paris. No dia 01/02/2019 ela irá partir.')
for entidade in doc.ents:
    print(entidade.text, entidade.label_)

Maria PER
Paris LOC


In [9]:
displacy.render(doc, style='ent', jupyter=True)

In [10]:
spacy.explain('PER')

'Named person or family.'

In [11]:
# treinar o modelo com os novos dados
!python train_new_entity_type.py -m pt -o modelo_kodama.md

Loaded model 'pt'
Losses {'ner': 28.863803359615147}
Losses {'ner': 32.39045933138531}
Losses {'ner': 11.391073097512754}
Losses {'ner': 26.009125748670613}
Losses {'ner': 24.092321053143678}
Losses {'ner': 22.271839798897318}
Losses {'ner': 22.498803939233767}
Losses {'ner': 19.13334190985188}
Losses {'ner': 24.209166664630175}
Losses {'ner': 22.854396803304553}
Losses {'ner': 19.715164292836562}
Losses {'ner': 21.714835434017004}
Losses {'ner': 20.43978180392878}
Losses {'ner': 13.057981245525298}
Losses {'ner': 20.726609552279115}
Losses {'ner': 17.836472530849278}
Losses {'ner': 17.474963728920557}
Losses {'ner': 16.901229339608108}
Losses {'ner': 19.732362221402582}
Losses {'ner': 16.18024340257398}
Losses {'ner': 18.116847230314306}
Losses {'ner': 16.17568147356178}
Losses {'ner': 14.888211632585637}
Losses {'ner': 20.093955073025427}
Losses {'ner': 15.860328152427378}
Losses {'ner': 15.769552267673134}
Losses {'ner': 13.79833963389683}
Losses {'ner': 22.195501245319065}
Losses {

In [12]:
nlp = spacy.load('./modelo_kodama.md')

In [13]:
doc = nlp('quero meu relatório de vendas entre 01/02/2020 e 01/03/2020')
displacy.render(doc, style='ent', jupyter=True)

Nós podemos observar que com um modelo probabislítico temos alguns problemas em identificar datas. Precisamos de mais exemplos e talvez nesse caso seja mais interessante utilizar uma abordagem baseda em regras (regex) e no caso de dados relativas como "ultimo mes" "semestre de 2020" podemos utilizar uma abordagem combinada com regras e bases de conhecimento

# Rule based and Knowledge Based

https://spacy.io/usage/rule-based-matching
https://spacy.io/usage/rule-based-matching#entityruler

In [14]:
from spacy.lang.pt import Portuguese
from spacy.pipeline import EntityRuler

nlp = Portuguese() # if you want to combina with existing prob model use spacy.load('pt')
ruler = EntityRuler(nlp)
patterns = [
    {
        'label': 'DATE', 'pattern': 
        [    # this will run per token and check the token sequence to match the rules
            {'TEXT' : {"REGEX": "[uú]ltimo"} },
            {'TEXT' : {"REGEX": "m[êe]s"} },
        ], 
        'id': 'date'
    },
    {
        'label': 'DATE', 'pattern': 
        [
            {'TEXT' : {"REGEX": "[uú]ltimo"} },
            {'TEXT' : {"REGEX": "\d"} },
            {'TEXT' : {"REGEX": "m[êe]s"} },
        ], 
        'id': 'date'
    },
    {
        'label': 'DATE', 'pattern': 
        [
            {'TEXT' : {"REGEX": "([Jj]aneiro|[Ff]evereiro|[Mm]ar[çc]o|[Aa]bril|[Mm]aio|[Jj]unho|[Jj]ulho|[Aa]gosto|[Ss]etembro|[Oo]utubro|[Nn]ovembro|[Dd]ezembro)"} }
        ], 
        'id': 'date'
    }, {
        'label': 'DATE', 'pattern': 
        [
            {'TEXT' : {"REGEX": "(hoje|ontem)"} }
        ], 
        'id': 'date'
    }
]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

messages = [
    'quero relatório de vendas do ultimo mês',
    'como foram as minhas vendas nos últimos 3 meses',
    'vendas do mês de agosto',
    'minhas vendas do mes de março até junho',
    'como foram as minhas vendas entre Maio e Agosto',
    'quanto vendi entre abril e junho?',
    'qual produto eu mais vendi hoje'
]

def test_date_entity(messages):
    for message in messages:
        doc = nlp(message)
        print([(ent.text, ent.label_) for ent in doc.ents])

test_date_entity(messages)

# nlp.to_disk("/path/to/model") saving model

[('ultimo mês', 'DATE')]
[('últimos 3 meses', 'DATE')]
[('agosto', 'DATE')]
[('março', 'DATE'), ('junho', 'DATE')]
[('Maio', 'DATE'), ('Agosto', 'DATE')]
[('abril', 'DATE'), ('junho', 'DATE')]
[('hoje', 'DATE')]
